# 4.1 - Procesos asíncronos


![async](images/async.png)




**[Documentación](https://docs.python.org/3/library/asyncio.html)**


**asyncio** es una biblioteca para escribir código [concurrente](https://es.wikipedia.org/wiki/Concurrencia_(inform%C3%A1tica)) utilizando la sintaxis async/await. Se utiliza como base en múltiples frameworks asíncronos de Python y provee un alto rendimiento en redes y servidores web, bibliotecas de conexión de base de datos, colas de tareas distribuidas, etc.

Suele encajar perfectamente para operaciones con límite de E/S y código de red estructurado de alto nivel. Además provee un conjunto de APIs de alto nivel para:

+ ejecutar corutinas de Python de manera concurrente y tener control total sobre su ejecución

+ realizar redes E/S y comunicación entre procesos(IPC)

+ controlar subprocesos

+ distribuir tareas a través de colas

+ sincronizar código concurrente

Adicionalmente, existen APIs de bajo nivel para desarrolladores de bibliotecas y frameworks para:

+ crear y administrar bucles de eventos, los cuales proveen APIs asíncronas para redes, ejecutando subprocesos, gestionando señales del sistema operativo, etc..

+ implementar protocolos eficientes utilizando transportes

+ bibliotecas puente basadas en retrollamadas y código con sintaxis async/wait

$$$$

Nosotros nos enfocaremos en el uso de bucles de eventos para la extracción de datos de la web.


### Hola Mundo

In [9]:
import asyncio

# funcion asincrona
async def saludar():
    print('¡Hola...')
    await asyncio.sleep(3)
    print('....Mundo!')
    print('Pero que pashhaaaaa....')
    
    
await saludar()        # en jupyter
#asyncio.run(main())   # en python .py

¡Hola...
....Mundo!
Pero que pashhaaaaa....


### Ejemplo response

Comprobando la respuesta de tres urls.

In [ ]:
import requests as req

In [ ]:
url='https://s3-eu-west-1.amazonaws.com/'

req.get(url)

In [ ]:
urls=[
    'https://s3-eu-west-1.amazonaws.com/ih-materials/uploads/data-static/documents/breakfast.jpg',
    'https://s3-eu-west-1.amazonaws.com/ih-materials/uploads/data-static/documents/forbidden',
    'https://s3-eu-west-1.amazonaws.com/ih-materials/uploads/data-static/documents/the-html5-breakfast-site.html'
]

In [ ]:
async def comprobar():
    
    bucle=asyncio.get_event_loop()  # voy a crear un bucle asincrono
    
    futuros=[bucle.run_in_executor(None, req.get, url) for url in urls] # bucle asincrono
    
    for res in await asyncio.gather(*futuros):  # aqui se los pido
        print(res.status_code)
        
await comprobar()

### Ejemplo ESPN

Volvamos al ejemplo de scrapeo de la págine de ESPN. Vamos a realizar múltiples requests para obtener los datos de todos los equipos.


https://www.espn.com/soccer/competitions

In [1]:
from selenium import webdriver

import time

import pandas as pd

from webdriver_manager.chrome import ChromeDriverManager

PATH=ChromeDriverManager().install()



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [/Users/iudh/.wdm/drivers/chromedriver/mac64_m1/100.0.4896.60/chromedriver] found in cache


In [2]:
url='https://www.espn.com/soccer/competitions'

# inicia el driver
driver=webdriver.Chrome(PATH)
driver.get(url)

# acepta cookies
aceptar=driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]')
aceptar.click()

time.sleep(2)

# selecciona equipos laliga
equipos=driver.find_element_by_css_selector('#fittPageContainer > div:nth-child(4) > div > div.layout__column.layout__column--1 > div > div:nth-child(3) > div:nth-child(1) > div > div:nth-child(4) > div > section > div > div > span:nth-child(2) > a')
equipos.click()


time.sleep(2)

equipos_stats_urls=driver.find_elements_by_css_selector('a.AnchorLink')

equipos_stats_urls=[e.get_attribute('href') for e in equipos_stats_urls if '/team/stats/' in e.get_attribute('href')]

driver.quit()

equipos_stats_urls


['https://www.espn.com/soccer/team/stats/_/id/96/alaves',
 'https://www.espn.com/soccer/team/stats/_/id/93/athletic-club',
 'https://www.espn.com/soccer/team/stats/_/id/1068/atletico-madrid',
 'https://www.espn.com/soccer/team/stats/_/id/83/barcelona',
 'https://www.espn.com/soccer/team/stats/_/id/85/celta-vigo',
 'https://www.espn.com/soccer/team/stats/_/id/3842/cadiz',
 'https://www.espn.com/soccer/team/stats/_/id/3751/elche',
 'https://www.espn.com/soccer/team/stats/_/id/88/espanyol',
 'https://www.espn.com/soccer/team/stats/_/id/2922/getafe',
 'https://www.espn.com/soccer/team/stats/_/id/3747/granada',
 'https://www.espn.com/soccer/team/stats/_/id/1538/levante',
 'https://www.espn.com/soccer/team/stats/_/id/84/mallorca',
 'https://www.espn.com/soccer/team/stats/_/id/97/osasuna',
 'https://www.espn.com/soccer/team/stats/_/id/101/rayo-vallecano',
 'https://www.espn.com/soccer/team/stats/_/id/244/real-betis',
 'https://www.espn.com/soccer/team/stats/_/id/86/real-madrid',
 'https://www

**Extracción asincrónica**

In [15]:
def asincrono(funcion):
    def eventos(*args, **kwargs):
        return asyncio.get_event_loop().run_in_executor(None, funcion, *args, **kwargs)
    return eventos

In [16]:
DATOS=[]
CABECERAS=[]

In [17]:
@asincrono
def extraer(url):
    
    global DATOS, CABECERAS
    
    # inicia el driver
    driver=webdriver.Chrome(PATH)
    driver.get(url)
    
    time.sleep(2)
    
    # acepta cookies
    aceptar=driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]')
    aceptar.click()

    time.sleep(2)


    disciplina=driver.find_element_by_xpath('//*[@id="fittPageContainer"]/div[2]/div[5]/div/div[1]/section/div/div[2]/nav/ul/li[2]/a')
    disciplina.click()

    time.sleep(2)

    tabla=driver.find_element_by_tag_name('tbody')

    filas=tabla.find_elements_by_tag_name('tr')


    datos=[[e.text for e in f.find_elements_by_tag_name('td')]+[url.split('/')[-1]] for f in filas]

    cabeceras=driver.find_element_by_tag_name('thead')

    cabeceras=[c.text for c in cabeceras.find_elements_by_tag_name('th')]+['TEAM']
    
    DATOS+=datos
    
    CABECERAS=cabeceras

In [18]:
for url in equipos_stats_urls[:3]:
    res=extraer(url)
    display(res)

Future exception was never retrieved
future: <Future finished exception=NoSuchElementException('no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="onetrust-accept-btn-handler"]"}\n  (Session info: chrome=100.0.4896.88)', None, None)>
Traceback (most recent call last):
  File "/Users/iudh/miniforge3/envs/clase/lib/python3.9/concurrent/futures/thread.py", line 52, in run
    result = self.fn(*self.args, **self.kwargs)
  File "/var/folders/fv/zc4lh9sn7bnf55fy6jhl0k900000gn/T/ipykernel_27193/878851846.py", line 11, in extraer
    aceptar=driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]')
  File "/Users/iudh/miniforge3/envs/clase/lib/python3.9/site-packages/selenium/webdriver/remote/webdriver.py", line 394, in find_element_by_xpath
    return self.find_element(by=By.XPATH, value=xpath)
  File "/Users/iudh/miniforge3/envs/clase/lib/python3.9/site-packages/selenium/webdriver/remote/webdriver.py", line 976, in find_element
    return self.exec

<Future pending cb=[_chain_future.<locals>._call_check_cancel() at /Users/iudh/miniforge3/envs/clase/lib/python3.9/asyncio/futures.py:384]>

<Future pending cb=[_chain_future.<locals>._call_check_cancel() at /Users/iudh/miniforge3/envs/clase/lib/python3.9/asyncio/futures.py:384]>

<Future pending cb=[_chain_future.<locals>._call_check_cancel() at /Users/iudh/miniforge3/envs/clase/lib/python3.9/asyncio/futures.py:384]>

In [19]:
#driver.quit()    # cuidado, necesitan cumplirse todas las promesas

In [21]:
df=pd.DataFrame(DATOS, columns=CABECERAS)

df.head()

,RK,NAME,P,YC,RC,PTS,TEAM
0,1,Geoffrey Kondogbia,22,9,1,12,atletico-madrid
1,2,José Giménez,20,8,0,8,atletico-madrid
2,,Stefan Savic,22,8,0,8,atletico-madrid
3,4,Daniel Wass,20,7,0,7,atletico-madrid
4,,Mario Hermoso,24,4,1,7,atletico-madrid


In [22]:
df.TEAM.unique()

array(['atletico-madrid', 'alaves', 'athletic-club'], dtype=object)

In [23]:
df.shape

(70, 7)